# Part 1: Extracting Chemical Named Entities

## 1. Obtaining Data

**ChemDNER Corpus v1.0**

The ChemDNER corpus consists of 10,000 PubMED abstracts and their corresponding label sets of named chemical entities. This data set is [publicly available](http://www.biocreative.org/resources/biocreative-iv/chemdner-corpus/) and can be downloaded directly using the shell script 

    load_data.sh

## 2. Loading Documents
For the extraction step, our goal is to get the highest recall as possible. In cases where we have labeled data, it's easy to get a recall estimate for our extraction pipline.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
import codecs
import operator
import itertools
from ddlite import *
from datasets import *
from utils import unescape_penn_treebank
from lexicons import AllUpperNounsMatcher, RuleTokenizedDictionaryMatch

In [2]:
parser = SentenceParser()
corpus = ChemdnerCorpus('datasets/chemdner_corpus/', parser=parser, 
                        cache_path="examples/cache/chemdner/")

# ChemNDER has pre-defined cross-validation folds -- use 100 docs
dev_set = sorted(corpus.cv["training"].keys())[:100]

# load training documents and collapse all sentences into a single list
documents = {doc_id:(corpus[doc_id]["sentences"],corpus[doc_id]["tags"]) for doc_id in dev_set}
sentences, gold_entities = zip(*documents.values())
sentences = list(itertools.chain.from_iterable(sentences))
gold_entities = list(itertools.chain.from_iterable(gold_entities))

# summary statistics
gold_entity_n = len(list(itertools.chain.from_iterable(gold_entities)))
word_n = sum([len(sent.words) for sent in sentences])
print("%d PubMed abstracts" % len(documents))
print("%d ChemNDER gold entities" % gold_entity_n)
print("%d tokens" % word_n)

100 PubMed abstracts
743 ChemNDER gold entities
21529 tokens


## 3. Building Matchers

The easiest way to identify candidates is through simple string matching using a dictionary of known entity names. Curating good lexicons can take some time, so we use pre-existing dictionaries provided by the *tmChem* tagger and a UMLS dictionary of all *Substance* semantic types (see the UMLS notebook for instructions how to create arbitrary dictionaries). The goal of matching is to acheive as high recall as possible. In real-world applications, we can't compute true recall, so it's important to try and get good coverage. 

- **DictionaryMatch** Match to an existing dictionary of known entity names.

- **RegexMatcher** Match words according to simple regular expressions. Here we just match Greek letters and  simple patterns of the form -3,4- which tend to indicate chemical names.

- **RuleTokenizedDictionaryMatch** Match a dictionary under a different tokenization scheme (in this case we provide a whitespace tokenizer. The resulting labels are mapped back into our primary CoreNLP token offset space.

- **AllUpperNounsMatcher** (From the Gene Tagger example) Identify all uppercase nouns in text. 

In [19]:
# tokenizer for matching within raw sentence text
def rule_tokenizer(s):
    s = re.sub("([,?!:;] )",r" \1",s)
    s = re.sub("([.]$)",r" .",s)
    return s.split()

# load dictionaries 
dict_fnames = ["datasets/dictionaries/chemdner/mention_chemical.txt",
               "datasets/dictionaries/chemdner/chebi.txt",
               "datasets/dictionaries/chemdner/addition.txt",
               "datasets/dictionaries/umls/substance-sab-all.txt"]

chemicals = []
for fname in dict_fnames:
    chemicals += [line.strip().split("\t")[0] for line in codecs.open(fname,"rU","utf-8").readlines()]

# remove stopwords
fname = "datasets/dictionaries/chemdner/stopwords.txt"
stopwords = {line.strip().split("\t")[0]:1 for line in open(fname,"rU").readlines()}
chemicals = {term:1 for term in chemicals if term not in stopwords}

# create matchers and extract candidates
extr1 = DictionaryMatch('C', chemicals, ignore_case=True)
extr2 = RuleTokenizedDictionaryMatch('C', chemicals, ignore_case=True, tokenizer=rule_tokenizer)
extr3 = RegexMatch('C',"[αβΓγΔδεϝζηΘθικΛλμνΞξοΠπρΣστυΦφχΨψΩω]+[-]+[A-Za-z]+", ignore_case=True)
extr4 = RegexMatch('C', "([-]*(\d[,]*)+[-])", ignore_case=True)
extr5 = RegexMatch('C',"[αβΔδη]+", ignore_case=True)
extr5a = RegexMatch('C',"((Ac|Ag|Al|Am|Ar|As|At|Au|B|Ba|Be|Bh|Bi|Bk|Br|C|Ca|Cd|Ce|Cf|Cl|Cm|Cn|Co|Cr|Cs|Cu|Db|Ds|Dy|Er|Es|Eu|F|Ga|Gd|Ge|H|He|Hf|Hg|Ho|Hs|I|In|Ir|K|Kr|La|Li|Lr|Lu|Lv|Md|Mg|Mn|Mo|Mt|N|Na|Nb|Nd|Ne|Ni|No|Np|O|Os|P|Pa|Pb|Pd|Pm|Po|Pr|Re|Rf|Rg|Rh|Rn|Ru|S|Sb|Sc|Se|Sg|Si|Sm|Sn|Sr|Ta|Tb|Tc|Te|Th|Ti|Tl|Tm|U|Uuo|Uup|Uus|Uut|V|W|Xe|Y|Yb|Zn|Zr)+([(]*\d+[+-]*[)]*)*)+\s", match_attrib="text", ignore_case=False)

extr5a = RegexMatch('C',"\d+\.\d+", match_attrib="text", ignore_case=True)

extr6 = AllUpperNounsMatcher('C')

In [20]:
matcher = extr5a#MultiMatcher(extr5a) #extr1, extr2, extr3, extr4, extr5, extr6)

## 4. Extracting Candidate Entities
Once we have matchers, we want to generate and store our candidate entity set for later use in learning. (Note this can take a long time, which is why you should precompute candidates before moving to the learning stage.)

In [21]:
candidates = Entities(sentences, matcher)

# Crude recall estimate (ignores actual span match)
mentions = [" ".join(unescape_penn_treebank([e.words[i] for i in e.idxs])) for e in candidates]
gold_mentions = list(zip(*itertools.chain.from_iterable(gold_entities))[0])

for m in mentions:
    print m
    if m in gold_mentions:
        gold_mentions.remove(m)
tp = gold_entity_n - len(gold_mentions)

print("Found %d candidate entities" % len(candidates))
print("Candidates: %.2f%% of all tokens" % (len(candidates)/float(word_n) * 100))
print("Annotations %.2f%% of all tokens" % (gold_entity_n/float(word_n) * 100))
print("~recall: %.2f (%d/%d)" % (float(tp) / gold_entity_n, tp, gold_entity_n))

candidates.dump_candidates("examples/cache/chem-candidates.pkl")

0.05 )
75.6 %
1p36 .23
40.69 %
41.99 %
37.94 %
93.7 %
32.3 %
20.4 %
40.4 %
1.2 -
1.3-fold
1.6-1
1.6-1 .9
0.5 %
1.3 %
1.76 %
1.2
0.04-1
0.04-1 .28 mgmL
0.08-2
0.08-2 .56 mg/mL
4.17-12
4.17-12 .01 ,
0.0001 )
0.001 )
0.001 )
7.5 %
7.5 %
7.5 %
7.5 %
7.5 %
0.05 )
0.009 )
0.05 )
0.3868
0.9573 mgmL
49.8
5.5 kD
49.8 ,
24.7 ,
21.1
5.5 kD
0.05 %
0.9
1.2
113.58
11.33
8.93
5.8
113.26
11.33
5.5
9.09
0.1 %
0.9989
0.9991 )
7.26-35
7.26-35 .75 gmL
2.20-11
2.20-11 .00 gmL
95.22-95
95.22-95 .83 .
10.2.2.0
10.2.2.0 (
10.2.2.0
10.2.2.0 (
18.10 %
15.68 %
14.38 %
159.34 C
10.79
150.69 C
9.0
0.4
3.8
0.37
130.6
2.52
3.9
2.6 ,
1.7-fold
47.54 %
3.45
0.07
3.8 ,
2.2 ,
5.4
3.8
5.4
3.1
0.1-30
1.10
1.10
2.5 ,
0.0001 )
1.9
63.2 M
9.6
12.1
59.3
81.7
0.083 MmL
0.02 %
0.02 %
0.02 %
0.5 mLmin
0.9993 )
4.18 %
5.05 %
2.08 %
93.82-102
93.82-102 .84 %
1.5 %
0.7 %
72.6 %
62.4 %
91.4 %
0.008
4.6
2.32
0.001 )
0.01 )
163.4 %
154.2 %
2.42-fold
10.8
4.2
2.3
1.0
6.7
2.1
81.5
26.7
14.8
5.8
2.7
0.8
5.6
0.9
95.4
44.2
Found 142 candida

In [32]:
candidates[23].render()

<!--Provide the canvas id (twice) and the words via python string formatting here--!>
 
 
 Acute effects of methiocarb on oxidative damage and the protective effects of vitamin E and taurine in the liver and kidney of Wistar rats .

### Error Analysis
For distant supervision, we want our candidate set to have high recall. We currently fall short for chemical named entities. If we look at the gold standard annotations, we can see why our string matching misses some entities. Note how paranthesis and other tokenization issues result in many missed entities. 

In [18]:
# What are we missing due to tokenization errors?
regex = ["[αβΓγΔδεϝζηΘθικΛλμνΞξοΠπρΣστυΦφχΨψΩω]+[-]+[A-Za-z]+"]
regex += ["([-]*(\d[,]*)+[-])"]
regex += ["[αβΔδη]+"]
regex = re.compile("(%s)" % "|".join(regex), re.I)

# regular expression matches
cache = {}
def regex_match(t):
    if t in cache:
        return cache[t]
    cache[t] = regex.search(t) != None
    return cache[t]

# simple term frequency count
def freq(terms):
    freqdist = {}
    for t in terms:
        if t not in freqdist:
            freqdist[t] = terms.count(t)
    return freqdist

tok_errors = freq([term for term in gold_mentions if term in chemicals or regex_match(term)])
oov_errors = freq([term for term in gold_mentions if term not in tok_errors])

print("Est. Tokenization Errors: %d" % (sum(tokenization_errors.values())))
print("Est. Out-of-vocabulary Errors: %d" % (sum(oov_errors.values())))

Est. Tokenization Errors: 83
Est. Out-of-vocabulary Errors: 202


We see that almost half our errors stem from tokenization issues. If we fixed all of those errors, we would have ~0.87 recall on the development set. If we actually look at OOV mentions we missed, we see there is considerable room for refining regular expressions to identify mentions like NaAsO(2) or FeSe, which just consist of element names. 

In [19]:
# print out our out of vocabulary terms
for term in sorted(oov_errors.items(),key=operator.itemgetter(1),reverse=1):
    print("%s: %d" % (term[0], oov_errors[term[0]]))

CO(2): 10
Al: 8
vitamin E: 7
RAP: 6
APC: 5
Glutamate: 5
GABA: 5
TCS: 4
H2 O2: 4
withanolide A: 4
GSH: 3
furostanol: 3
cAMP: 3
LY541850: 3
L-AP4: 3
DHPG: 3
Schiff bases: 3
PLA: 3
steroidal saponins: 3
vicinal diol: 2
THF: 2
Fe(II): 2
boeravinone E: 2
O: 2
S: 2
boeravinone B: 2
MPTP: 2
Glaucogenin E: 2
Ser: 2
C-geranylated flavonoids: 2
mimulone B: 2
BBH: 2
pyrimidinedione: 2
MTT: 2
STZ: 2
Mercury: 1
decanoate salt: 1
Tomentomimulol: 1
PAH: 1
Anthocyanins: 1
arachidonic (C20 : 4ω-6) and eicosapentaenoic (C20 : 5ω-3) acids: 1
Rapamycin: 1
F-IMC: 1
cyclododeca[d]oxazole: 1
phosphoinositol(3,5)P2: 1
Saturated fatty acids: 1
Polycyclic aromatic hydrocarbons: 1
ammonium-hexachloroplatinate: 1
oxygenated monoterpenes: 1
tocochromanols: 1
(1)H: 1
TBBPA: 1
Methiocarb: 1
ixoroside: 1
Poloxamer 407: 1
Azole: 1
DPPH: 1
betulin aldehyde: 1
Polymethoxylated flavones: 1
poly (ADP-ribose): 1
MTEP: 1
Berberine hydrochloride: 1
NaOH: 1
cinnamic-alcohol: 1
steroidal glycosides: 1
SiO2: 1
paraphenylendiami

### Example Best-in-class Tagger

The winning system in the 2013 BioCreative IV CHEMDNER task was tmChem which used 2 linear chain conditional random fields (CRF) with different tokenziation approaches and feature sets.

| Model       | Precision | Recall | F1     |
|-------------|-----------|--------|--------|
| Model 1     | 0.8595    | 0.8721 | 0.8657 |
| Model 2     | **0.8909**    | 0.8575 | **0.8739** |
| Heuristic Combination     | 0.8516    | 0.8906 | 0.8706 |
| Highest Recall | 0.7672    | **0.9212** | 0.8372 |

Leaman, Robert, Chih-Hsuan Wei, and Zhiyong Lu. ["tmChem: a high performance approach for chemical named entity recognition and normalization."](http://www.ncbi.nlm.nih.gov/pmc/articles/PMC4331693/) J. Cheminformatics 7.S-1 (2015): S3.